# XGBoost from scratch

- http://explained.ai/gradient-boosting/index.html
- https://medium.com/mlreview/gradient-boosting-from-scratch-1e317ae4587d
- https://towardsdatascience.com/boosting-algorithm-xgboost-4d9ec0207d